**Oscilloscope Interface**
    For the Picoscope please download the PicoSDK for the Picoscope 3000 3026D from https://www.picotech.com/downloads

In [3]:
import ctypes
import pyvisa as visa 
import chipwhisperer as cw
from picosdk.ps3000a import ps3000a as ps

class Scope: 
    
    # init scope with a given scope IP
    def __init__(self, scope_type, scope_ip):
        self.scope_type = scope_type
        
        if self.scope_type == "Lecroy":
            self.visa = visa.ResourceManager()
            self.oscilloscope = self.visa.open_resource(scope_ip)
            self.oscilloscope.timeout = 5000 # indicated in manual 
            self.oscilloscope.clear()
            
        elif self.scope_type == "CW":
            self.oscilloscope = cw.scope()
            self.target = cw.target(self.oscilloscope)
            
        elif self.scope_type == "Pico":
            self.status = {}
            self.chandle = ctypes.c_int16()
            self.status["openunit"] = ps.ps3000aOpenUnit(ctypes.byref(self.chandle), None)

        
    # close open scope    
    def close(self):
        if self.scope_type == "Lecroy":
            self.oscilloscope.close()
        elif self.scope_type == "CW":
            self.oscilloscope.disconnect()
    
    # configure the scope, code was adapted from Scope.ipynb
    def configure_scope(self, channel, v_res, timescale, fs, duration, voffset):
        if self.scope_type == "LeCroy":        
            # clear scope and get ready for use
            self.oscilloscope.write(channel + ":TRA ON")
            self.oscilloscope.write(r"""vbs 'app.measure.clearall' """)
            self.oscilloscope.write(r"""vbs 'app.Acquisition.ClearSweeps' """)
            
            self.oscilloscope.write("TDIV " + timescale)
            self.oscilloscope.write(channel + ":VDIV " + v_res)
            self.oscilloscope.write(r"""vbs 'app.Acquisition.Horizontal.Maximize = "FixedSampleRate" '""")
            self.oscilloscope.write(r"""vbs 'app.Acquisition.Horizontal.SampleRate = "%s" '""" %fs)
            self.oscilloscope.write(r"""vbs 'app.Acquisition.Horizontal.AcquisitionDuration = "%s" '"""%duration)
            self.oscilloscope.write(r"""vbs 'app.Acquisition.C3.VerOffset = "%s" '""" % voffset)
        elif self.scope_type == "CW":
            print("TODO: CW Lite")
            
    #set the trigger source and type
    def set_trigger(self, source, trigger_type):
        self.oscilloscope.write(r"""vbs 'app.acquisition.triggermode = "%s" '"""%trigger_type)
        

In [ ]:
scope = Scope("Pico", "")
print(scope.status)